In [1]:
import pandas as pd
import numpy as np
import datetime
import os

print(datetime.datetime.now())
os.getcwd()

2020-02-03 13:40:42.143344


'/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Planner_Request/JT_zips_by_SOTF_Density_20200128'

In [2]:
df_store_type=pd.ExcelFile("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/other_input/BL Store List Report from Dom 12.20.19.xlsx")
df_store_type=df_store_type.parse("Store List",dtype=str)
df_store_type=df_store_type[["Store",'SOTF']]
df_store_type=df_store_type[df_store_type['SOTF']=="Yes"]
df_store_type=df_store_type[df_store_type['Store']!="TBD"]
print(len(df_store_type),df_store_type['Store'].nunique())
df_store_type=df_store_type.rename(columns={"Store":"location_id","SOTF":"Store_Type"})
df_store_type['Store_Type']=df_store_type['Store_Type'].replace({"Yes":"SOTF"})
print(df_store_type['Store_Type'].unique())
df_store_type.head(2)

479 479
['SOTF']


,location_id,Store_Type
0,1,SOTF
8,49,SOTF


In [3]:
store_list=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20200101-135601-098.txt",
                        dtype=str,sep="|",usecols=['location_id'])
store_list=store_list.drop_duplicates()
store_list=pd.merge(store_list,df_store_type,on="location_id",how="left")
store_list['Store_Type']=store_list['Store_Type'].fillna("Legacy")
store_list=store_list[store_list['location_id']!="145"]
store_list=store_list[store_list['location_id']!="6990"]


In [4]:
df_store_density=pd.read_excel("/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Planner_Request/JT_zips_by_SOTF_Density_20200128/BL 2020Q1 Budget Allocation Model 20200122 BR Summary.xlsx",
                              sheet_name="Full Store List Q1",dtype=str,usecols=['Store #','Density Tier','SOTF'])
df_store_density=df_store_density.rename(columns={"Store #":"location_id","Density Tier":"density","SOTF":"SOTF_in_density_file"})
df_store_density['SOTF_in_density_file']=df_store_density['SOTF_in_density_file'].replace({"NonSOTF":"Legacy"})
df_store_density.shape

(1412, 3)

In [5]:
store_list=pd.merge(store_list,df_store_density,on="location_id",how="outer")
store_list=store_list[pd.notnull(store_list['location_id'])]
store_list.shape

(1420, 4)

In [6]:
df_different_store_type=store_list[store_list['Store_Type']!=store_list['SOTF_in_density_file']]
print(df_different_store_type.shape)

df_missing_density=store_list[pd.isnull(store_list['density'])]
print(df_missing_density.shape)


(32, 4)
(14, 4)


In [7]:
writer=pd.ExcelWriter("./to_confirm_with_BR_inconsistence_store_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_different_store_type.to_excel(writer,"different_store_type",index=False)
df_missing_density.to_excel(writer,"missing_density",index=False)
writer.save()

In [8]:
# Confirmed with BR through email RE: Big Lots Q1 Zip Code Lists, Wednesday, January 29, 2020 at 4:35 PM
# Use Dom's SOTF and exclude the denstiy missing stores

del store_list['SOTF_in_density_file']
store_list=store_list[pd.notnull(store_list['density'])]
print(store_list['Store_Type'].unique())
print(store_list['density'].unique())
print(store_list.shape)
store_list.head(2)


['SOTF' 'Legacy']
['M' 'L' 'H']
(1406, 3)


,location_id,Store_Type,density
0,1,SOTF,M
1,29,Legacy,M


In [10]:
store_list.groupby(['Store_Type','density'])['location_id'].nunique().to_frame().reset_index()

,Store_Type,density,location_id
0,Legacy,H,135
1,Legacy,L,9
2,Legacy,M,798
3,SOTF,H,6
4,SOTF,L,133
5,SOTF,M,325


In [9]:
TA_excel=pd.ExcelFile("/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Planner_Request/Ramiro_TA_request_20200124/BL_zips_in_TA_2020_updated_30_new_stores_JL_2020-01-27.xlsx")
print(TA_excel.sheet_names)

['unique_zips', 'view_by_TA', 'view_by_store', 'view_by_store_and_zip', 'store_list_full_202001']


In [10]:
df_zips_by_store=TA_excel.parse("view_by_store_and_zip",dtype=str)
df_zips_by_store=df_zips_by_store[['location_id','zip_cd','zip_type']]

df_store_list_with_zip=pd.merge(store_list,df_zips_by_store,on="location_id",how="left")
df_store_list_with_zip.shape

(23383, 5)

In [12]:
df_store_list_with_zip

,location_id,Store_Type,density,zip_cd,zip_type
0,1,SOTF,M,43227,trans_P
1,1,SOTF,M,43207,trans_P
2,1,SOTF,M,43209,trans_P
3,1,SOTF,M,43232,trans_P
4,1,SOTF,M,43125,trans_S
...,...,...,...,...,...
23378,5410,SOTF,L,45263,zips_10
23379,5410,SOTF,L,45264,zips_10
23380,5410,SOTF,L,45267,zips_10
23381,5410,SOTF,L,45268,zips_10


In [13]:
df_EASI_income=pd.read_csv("/home/jian/Docs/EASI/2018-07-19/ZIP4_18_DATA_A2_CSV/ZIP4_18_DATA_A2.CSV",
                           dtype=str,skiprows=1,usecols=['ZIP_CODE','ZIP4','HH18','AVGHHINC'])
df_EASI_income['HH18']=df_EASI_income['HH18'].astype(float)
df_EASI_income['AVGHHINC']=df_EASI_income['AVGHHINC'].astype(float)
df_EASI_income['Total_Inc']=df_EASI_income['HH18']*df_EASI_income['AVGHHINC']
df_EASI_income=df_EASI_income.groupby("ZIP_CODE")['HH18','Total_Inc'].sum().reset_index()
df_EASI_income['Avg_Inc_HH']=df_EASI_income['Total_Inc']/df_EASI_income['HH18']
df_EASI_income=df_EASI_income.rename(columns={"ZIP_CODE":"zip_cd"})
df_EASI_income=df_EASI_income[['zip_cd','Avg_Inc_HH']]

df_EASI_income.shape

(39602, 2)

In [15]:
df_store_list_with_zip=pd.merge(df_store_list_with_zip,df_EASI_income,on="zip_cd",how="left")
df_store_list_with_zip.head(2)

,location_id,Store_Type,density,zip_cd,zip_type,Avg_Inc_HH
0,1,SOTF,M,43227,trans_P,59889.822133
1,1,SOTF,M,43207,trans_P,59020.889831


In [16]:
writer=pd.ExcelWriter("./BL_zips_all_stores_in_detail_and_Density_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_store_list_with_zip.to_excel(writer,"details_by_store",index=False)
writer.save()

In [38]:
writer=pd.ExcelWriter("./BL_zips_by_SOTF_Density_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
dict_unique_zips={}
max_len=0
for seg,df_group in df_store_list_with_zip.groupby(['Store_Type','density']):

    segment_name="_".join(seg)
    df_unique_zip_cd=df_group[['zip_type','zip_cd','Avg_Inc_HH']].drop_duplicates()
    df_unique_zip_cd=df_unique_zip_cd.sort_values(['zip_type','zip_cd']).drop_duplicates("zip_cd")
    df_group['Store_Density_group']=segment_name
    dict_unique_zips.update({segment_name:df_unique_zip_cd})
    
    df_group.to_excel(writer,"by_store_"+segment_name,index=False)
    
df_all_SOTF_zips_detail=df_store_list_with_zip[df_store_list_with_zip['Store_Type']=="SOTF"]
df_all_SOTF_zips_detail['Store_Density_group']="SOTF_all"
df_all_SOTF_zips_detail.to_excel(writer,"by_store_SOTF_all",index=False)

df_all_SOTF_zips_unique=df_all_SOTF_zips_detail[['zip_type','zip_cd','Avg_Inc_HH']].drop_duplicates()
df_all_SOTF_zips_unique=df_all_SOTF_zips_unique.sort_values(['zip_type','zip_cd']).drop_duplicates("zip_cd")
dict_unique_zips.update({"SOTF_all":df_all_SOTF_zips_unique})

if len(list_all_SOTF_zips_unique)>max_len:
    max_len=len(list_all_SOTF_zips_unique)
        

/home/jian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/home/jian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [41]:
for k,v in dict_unique_zips.items():
    df_fill=pd.DataFrame({"zip_type":[np.nan]*(max_len-len(v)),"zip_cd":[np.nan]*(max_len-len(v)),"Avg_Inc_HH":[np.nan]*(max_len-len(v))})
    new_v=v.append(df_fill)
    new_v.to_excel(writer,"unique_"+k,index=False)


writer.save()

In [42]:
os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Planner_Request/JT_zips_by_SOTF_Density_20200128'

In [43]:
df=pd.read_table("/home/jian/BigLots/2020_by_weeks/MediaStorm_2020-01-18/MediaStormDailySales20200121-111749-649.txt",
                dtype=str,nrows=10,sep="|")

In [44]:
df

,location_id,transaction_dt,transaction_id,customer_id_hashed,class_code_id,subclass_id,item_id,item_transaction_units,item_transaction_amt
0,1027,2020-01-18,2108,51cc9db6c31bf222baea4e50b9a6b1be2c7c4ec833bcfe...,14001,1,140008972,1,00000000000001.6000
1,1960,2020-01-14,9803,NaN,55006,1,810330818,-1,-00000000000002.5000
2,5353,2020-01-15,9988,NaN,61501,3,810435268,1,00000000000299.9900
3,5269,2020-01-18,7147,3fb7f19d961a20055d0fb89d584bd36b35f26c56e29e32...,12007,4,810358789,1,00000000000002.8000
4,4516,2020-01-13,9006,4f819d6299f6b7c14fe1bafcba4edfebbb7db239e6b091...,35016,14,810373236,1,00000000000036.9900
5,5234,2020-01-15,4697,NaN,11404,4,810447658,1,00000000000001.5000
6,4081,2020-01-13,8480,0e4a8229e683bd22cf40d646f4164e848355b9579b2968...,12002,2,810360230,1,00000000000001.0000
7,5331,2020-01-18,2651,NaN,13003,3,810333801,1,00000000000003.2000
8,509,2020-01-12,7774,NaN,21013,2,810412855,3,00000000000104.3100
9,4002,2020-01-16,9005,7e3b4038eb960c824005600f57cff9b9afe68deba1bb19...,61501,3,810435268,1,00000000000299.9900


In [45]:
df_prod_taxo=pd.read_table("/home/jian/BigLots/static_files/ProductTaxonomy/MediaStormProductTaxonomy20200101-135600-916.txt",
                          dtype=str,nrows=10,sep="|")

In [46]:
df_prod_taxo

,division_id,department_id,class_code_id,subclass_id,subclass_desc
0,1,108,11001,2,Non-Chocolate Bagged
1,1,108,11001,4,Single Non-Chocolate
2,1,108,11001,6,King Size Non-Chocolate
3,1,108,11001,8,Count Goods Non-Chocolate
4,1,108,11001,10,Gum And Mints
5,1,108,11001,12,Big Top Checkstand
6,1,108,11001,14,Mint & Breath Checkstand
7,1,108,11002,1,Valentine
8,1,108,11002,3,Halloween
9,1,108,11002,999,Finance Use Only
